In [1]:
import os
from pathlib import Path

import numpy as np
import tensorflow as tf

try:
    from keras.layers.merge import _Merge as Merge
except Exception:
    from keras.layers.merging.base_merge import _Merge as Merge

from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import math_ops

import hls4ml
from hls4ml.converters.keras_to_hls import parse_default_keras_layer
from hls4ml.model.attributes import ConfigurableAttribute, TypeAttribute
from hls4ml.model.types import FixedPrecisionType, RoundingMode, SaturationMode

2024-10-10 19:05:49.221992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 19:05:49.348681: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 19:05:49.351704: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-10 19:05:49.351719: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

/home/aelabd/RHEED/hls4ml/hls4ml/converters/__init__.py:29: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


## Keras frontend

In [2]:
# Keras implementation of a KL layer
class KLLoss(Merge):
    '''Keras implementation of a KL loss custom layer'''

    @tf_utils.shape_type_conversion
    def build(self, input_shape):
        super().build(input_shape)

    def _merge_function(self, inputs):
        mean = inputs[0]
        log_var = inputs[1]

        kl = 1.0 + log_var - math_ops.square(mean) - math_ops.exp(log_var)
        kl = -0.5 * math_ops.reduce_mean(kl, axis=-1, keepdims=True)

        return kl

## hls4ml frontend

In [3]:
# hls4ml implementations
class HKLLoss(hls4ml.model.layers.Layer):
    '''hls4ml implementation of a KL loss custom layer'''

    _expected_attributes = [
        ConfigurableAttribute('table_size', default=1024),
        ConfigurableAttribute('exp_range', default=8),
        TypeAttribute('accum'),
        TypeAttribute(
            'sum',
            default=FixedPrecisionType(18, 8, rounding_mode=RoundingMode.RND, saturation_mode=SaturationMode.SAT),
        ),
        TypeAttribute(
            'exp_table',
            default=FixedPrecisionType(18, 8, rounding_mode=RoundingMode.RND, saturation_mode=SaturationMode.SAT),
        ),
    ]

    def initialize(self):
        self.add_output_variable(shape=[1], dim_names=[f'KL_LOSS_{self.index}'])

# Templates
distance_config_template = """struct config{index} : nnet::distance_config {{
    static const unsigned n_in = {n_in};
    static const unsigned n_out = 1;
    typedef {accum_t.name} accum_t;
    typedef {sum_t.name} sum_t;
    typedef {exp_table_t.name} exp_table_t;
    static const unsigned table_size = {table_size};
    static constexpr float exp_range = {exp_range};
}};\n"""
distance_function_template = 'nnet::klloss<{input1_t}, {input2_t}, {output_t}, {config}>({input1}, {input2}, {output});'
distance_include_list = ['nnet_utils/kl_layer.h']

class HKLLossConfigTemplate(hls4ml.backends.template.LayerConfigTemplate):
    def __init__(self):
        super().__init__(HKLLoss)
        self.template = distance_config_template

    def format(self, node):
        params = self._default_config_params(node)
        params['n_in'] = node.get_input_variable(node.inputs[0]).shape[0]
        params['n_out'] = 1
        return self.template.format(**params)
    
class HKLLossFunctionTemplate(hls4ml.backends.template.FunctionCallTemplate):
    def __init__(self):
        super().__init__(HKLLoss, include_header=distance_include_list)
        self.template = distance_function_template

    def format(self, node):
        params = {}
        params['config'] = f'config{node.index}'
        params['input1_t'] = node.get_input_variable(node.inputs[0]).type.name
        params['input2_t'] = node.get_input_variable(node.inputs[1]).type.name
        params['output_t'] = node.get_output_variable().type.name
        params['input1'] = node.get_input_variable(node.inputs[0]).name
        params['input2'] = node.get_input_variable(node.inputs[1]).name
        params['output'] = node.get_output_variable().name

        return self.template.format(**params)
    
# Parser for converter
def parse_klloss_layer(keras_layer, input_names, input_shapes, data_reader):
    assert 'KLLoss' in keras_layer['class_name']

    layer = parse_default_keras_layer(keras_layer, input_names)
    output_shape = [input_shapes[0][0], 1]

    return layer, output_shape



## Run test

In [4]:
def make_models():
    backend = hls4ml.backends.get_backend('Vivado')
    try:
        # Register the converter for custom Keras layer
        hls4ml.converters.register_keras_layer_handler('KLLoss', parse_klloss_layer)

        # Register the hls4ml's IR layer
        hls4ml.model.layers.register_layer('KLLoss', HKLLoss)

        # Register the optimization passes (if any)
        

        # Register template passes for the given backend
        backend.register_template(HKLLossConfigTemplate)
        backend.register_template(HKLLossFunctionTemplate)
    except Exception:
        pass

    # Register HLS implementation
    # p =  os.getcwd() + '/kl_layer.h'
    # p = Path(__file__).parent / 'kl_layer.h'
    p =  Path(os.getcwd())/'kl_layer.h'
    backend.register_source(p)

    # Test if it works
    # Create a dummy Keras model with KL loss layer
    inp = tf.keras.layers.Input(shape=(19, 3, 1))
    z_mean = tf.keras.layers.Dense(10)(inp)
    z_log_var = tf.keras.layers.Dense(10)(inp)
    custom_output = KLLoss()([z_mean, z_log_var])
    # create new model
    kmodel = tf.keras.models.Model(inputs=inp, outputs=custom_output)
    kmodel.summary()

    # test on random inputs
    x = np.random.randint(-5, 5, (1, 19, 3, 1), dtype='int32')
    kres = kmodel(x)

    # Create dummy config
    config = {}
    config['Model'] = {
        'Precision': 'ap_fixed<16,6>',
        'ReuseFactor': 1,
        'ParallelizationFactor': 1,
        'Strategy': 'Resource',
    }
    hmodel = hls4ml.converters.convert_from_keras_model(
        kmodel,
        output_dir='hls4mlprj_kl_layer',
        backend='Vivado',
        io_type='io_parallel',
        part='xcvu9p-flga2577-2-e',
        hls_config=config,
    )

    hmodel.compile()
    return kmodel, hmodel


In [5]:
k_model, hls_model = make_models()

2024-10-10 19:05:52.913762: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-10 19:05:52.914024: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-10-10 19:05:52.914151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-10-10 19:05:52.914225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-10-10 19:05:52.914297: W tensorf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 19, 3, 1)]   0           []                               
                                                                                                  
 dense (Dense)                  (None, 19, 3, 10)    20          ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 19, 3, 10)    20          ['input_1[0][0]']                
                                                                                                  
 kl_loss (KLLoss)               (None, 19, 3, 1)     0           ['dense[0][0]',                  
                                                                  'dense_1[0][0]']            

In [6]:
x = np.random.randint(-5, 5, (1, 19, 3, 1), dtype='int32')
kres = k_model(x)
hres = hls_model.predict(x.astype('float32'))

In [7]:
kres

<tf.Tensor: shape=(1, 19, 3, 1), dtype=float32, numpy=
array([[[[ 3.2514224 ],
         [ 2.0610871 ],
         [-0.        ]],

        [[ 1.4704463 ],
         [ 2.0610871 ],
         [ 1.1571666 ]],

        [[ 1.4704463 ],
         [ 2.0610871 ],
         [ 3.2514224 ]],

        [[ 1.4704463 ],
         [ 2.0610871 ],
         [ 0.14069606]],

        [[ 2.9574256 ],
         [-0.        ],
         [ 2.9574256 ]],

        [[ 2.9574256 ],
         [ 0.5174905 ],
         [ 2.9574256 ]],

        [[ 3.2514224 ],
         [ 1.1571666 ],
         [ 0.5174905 ]],

        [[-0.        ],
         [ 0.13138631],
         [ 1.1571666 ]],

        [[ 2.0610871 ],
         [ 0.5984329 ],
         [ 1.4704463 ]],

        [[ 2.0610871 ],
         [ 2.9574256 ],
         [ 0.5174905 ]],

        [[ 0.5984329 ],
         [ 0.5174905 ],
         [ 0.5984329 ]],

        [[-0.        ],
         [ 2.0610871 ],
         [ 2.9574256 ]],

        [[ 3.2514224 ],
         [ 3.2514224 ],
         

In [8]:
hres

array([0.8261719], dtype=float32)

In [9]:
print(kres - hres)

tf.Tensor(
[[[[ 2.4252505 ]
   [ 1.2349153 ]
   [-0.8261719 ]]

  [[ 0.6442745 ]
   [ 1.2349153 ]
   [ 0.33099473]]

  [[ 0.6442745 ]
   [ 1.2349153 ]
   [ 2.4252505 ]]

  [[ 0.6442745 ]
   [ 1.2349153 ]
   [-0.6854758 ]]

  [[ 2.1312537 ]
   [-0.8261719 ]
   [ 2.1312537 ]]

  [[ 2.1312537 ]
   [-0.30868137]
   [ 2.1312537 ]]

  [[ 2.4252505 ]
   [ 0.33099473]
   [-0.30868137]]

  [[-0.8261719 ]
   [-0.6947856 ]
   [ 0.33099473]]

  [[ 1.2349153 ]
   [-0.22773898]
   [ 0.6442745 ]]

  [[ 1.2349153 ]
   [ 2.1312537 ]
   [-0.30868137]]

  [[-0.22773898]
   [-0.30868137]
   [-0.22773898]]

  [[-0.8261719 ]
   [ 1.2349153 ]
   [ 2.1312537 ]]

  [[ 2.4252505 ]
   [ 2.4252505 ]
   [ 1.2349153 ]]

  [[-0.6947856 ]
   [-0.30868137]
   [ 0.33099473]]

  [[-0.6947856 ]
   [-0.22773898]
   [-0.30868137]]

  [[-0.22773898]
   [-0.6947856 ]
   [-0.30868137]]

  [[-0.22773898]
   [ 1.2349153 ]
   [-0.22773898]]

  [[ 1.2349153 ]
   [-0.22773898]
   [ 0.33099473]]

  [[ 0.6442745 ]
   [ 0.6442745 ]
 